# Introduction

My report is for those who are planning to live in Montreal in Canada and who don't know in which neighbourhood they had to move.  I'll try to provide some informations on this with all factors to understand how to start this new venture.

# Neighborhood Problem

In my report I'm going to focus on problems one might face while moving in a new city. Maybe you need to know the venues in the neighbourhood. On what basis can you decide where you're going to live? While selecting the place there are key points to consider like if you need to have a good restaurants for dinner or a gym to work out.




In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
#SCIKIT Learn librairies
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score

def cross_validate(model, n_splits = 10):
    
    k_fold = KFold(n_splits = n_splits)
    scores = [model.fit(X[train], y[train]).score(X[test], y[test]) for train, test in k_fold.split(X)]
    
    scores = np.percentile(scores, [40, 50, 60])
    return scores

# Import and process the Montreal Neighborhood DataSet

This dataset reports the Borough and Neighborhood in Montreal. Data is extracted from the Montreal City's website (http://donnees.ville.montreal.qc.ca/dataset/f38c91a1-e33f-4475-a112-3b84b1c60c1e/resource/a80e611f-5336-4306-ba2a-fd657f0f00fa/download/quartierreferencehabitation.geojson) system.




In [3]:
!wget -q -O 'montreal_data.json' http://donnees.ville.montreal.qc.ca/dataset/f38c91a1-e33f-4475-a112-3b84b1c60c1e/resource/a80e611f-5336-4306-ba2a-fd657f0f00fa/download/quartierreferencehabitation.geojson
print('Data downloaded!')

Data downloaded!


In [4]:
with open('montreal_data.json') as json_data: #open the data
    montreal_data = json.load(json_data)

In [5]:
montreal_data #print the data

{'type': 'FeatureCollection',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'no_qr': '53',
    'nom_qr': 'Rivière-des-Prairies',
    'no_arr': '33',
    'nom_arr': 'Rivière-des-Prairies–Pointe-aux-Trembles',
    'nom_mun': 'Montréal'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-73.54470817788885, 45.64824326107301],
       [-73.54515773817808, 45.64801297804962],
       [-73.54834058012187, 45.646452856543355],
       [-73.54884681377214, 45.646203906855426],
       [-73.55246117333564, 45.644411218599714],
       [-73.55320519665015, 45.64400267644135],
       [-73.55395617306405, 45.64357351823839],
       [-73.55454848320028, 45.643210754136135],
       [-73.5555002142304, 45.64261770646879],
       [-73.55623787430996, 45.64213463514684],
       [-73.55684080880785, 45.641739637131685],
       [-73.5574385537578, 45.64134814815744],
       [-73.55990600064828, 45.63973190

In [6]:
neighborhoods_data = montreal_data['features'] #select a part of the json file

In [7]:
neighborhoods_data[0]

{'type': 'Feature',
 'properties': {'no_qr': '53',
  'nom_qr': 'Rivière-des-Prairies',
  'no_arr': '33',
  'nom_arr': 'Rivière-des-Prairies–Pointe-aux-Trembles',
  'nom_mun': 'Montréal'},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-73.54470817788885, 45.64824326107301],
     [-73.54515773817808, 45.64801297804962],
     [-73.54834058012187, 45.646452856543355],
     [-73.54884681377214, 45.646203906855426],
     [-73.55246117333564, 45.644411218599714],
     [-73.55320519665015, 45.64400267644135],
     [-73.55395617306405, 45.64357351823839],
     [-73.55454848320028, 45.643210754136135],
     [-73.5555002142304, 45.64261770646879],
     [-73.55623787430996, 45.64213463514684],
     [-73.55684080880785, 45.641739637131685],
     [-73.5574385537578, 45.64134814815744],
     [-73.55990600064828, 45.63973190576037],
     [-73.56524685477271, 45.63623175565011],
     [-73.56524738439809, 45.63623229529295],
     [-73.56837254638758, 45.63914167206348],
     [-73.5701836093

In [8]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [9]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [10]:
for data in neighborhoods_data: #file the dataframe
    borough = neighborhood_name = data['properties']['nom_arr'] 
    neighborhood_name = data['properties']['nom_qr']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[0][0][0][1]
    neighborhood_lon = neighborhood_latlon[0][0][0][0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [11]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Rivière-des-Prairies–Pointe-aux-Trembles,Rivière-des-Prairies,45.648243,-73.544708
1,Mercier–Hochelaga-Maisonneuve,Beaurivage,45.589751,-73.523954
2,Mercier–Hochelaga-Maisonneuve,Tétreaultville,45.612298,-73.516443
3,Mercier–Hochelaga-Maisonneuve,Dupéré,45.604545,-73.537125
4,Mercier–Hochelaga-Maisonneuve,Guybourg,45.576035,-73.529675


In [12]:
#print how many boroughs and neighborhoods Montreal has.
print('The dataframe has {} boroughs and {} neighborhoods.'.format( 
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 20 boroughs and 91 neighborhoods.


In [13]:
address = 'Montreal, MTL' #Print the coordinate of Montreal

geolocator = Nominatim(user_agent="mtl_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Montreal are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Montreal are 45.5332373, -73.5671463.


# Print the data through a map

We wanted to print the neighborhoods on a map and then see the venues into it!

In [14]:
# create map of Montreal using latitude and longitude values to see the neighborhoods
map_montreal = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_montreal)  
    
map_montreal

In [15]:
#Foursquare API code
CLIENT_ID = 'GWESFZAVSTZRZGUIPVBDOY40HKO2JB32HCLP0PFUNFIV253F' 
CLIENT_SECRET = 'TMWRDBQ4FLRK0UJ1CUVEWPL22FOB2UZ2WYHRU2QUNKSSEXCA'
VERSION = '20180604'
LIMIT = 100
radius= 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)



Your credentails:
CLIENT_ID: GWESFZAVSTZRZGUIPVBDOY40HKO2JB32HCLP0PFUNFIV253F
CLIENT_SECRET:TMWRDBQ4FLRK0UJ1CUVEWPL22FOB2UZ2WYHRU2QUNKSSEXCA


In [16]:
#print the url
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=GWESFZAVSTZRZGUIPVBDOY40HKO2JB32HCLP0PFUNFIV253F&client_secret=TMWRDBQ4FLRK0UJ1CUVEWPL22FOB2UZ2WYHRU2QUNKSSEXCA&v=20180604&ll=45.5332373,-73.5671463&radius=500&limit=100'

In [17]:
#print the values for the first neighborhood
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rivière-des-Prairies are 45.64824326107301, -73.54470817788885.


In [18]:
#Send the GET request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d386a2edb1d81003917dd04'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Plateau-Mont-Royal',
  'headerFullLocation': 'Plateau-Mont-Royal, Montreal',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 30,
  'suggestedBounds': {'ne': {'lat': 45.537737304500006,
    'lng': -73.56073426282813},
   'sw': {'lat': 45.5287372955, 'lng': -73.57355833717187}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c06cf964a52031fa20e3',
       'name': 'Maamm Bolduc',
       'location': {'address': '4351 ave. de Lorimier',
        'lat': 45.53424305181713,
        'lng': -73.570160933419,
        'labeledLatLngs': [{'label': 'display',
     

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Maamm Bolduc,American Restaurant,45.534243,-73.570161
1,Centre Père Sablon,Gym,45.531192,-73.570652
2,Animal Vert l'Épicerie pour animaux,Pet Store,45.530684,-73.566262
3,Parc Baldwin,Park,45.536330,-73.566510
4,Les Belles Soeurs,Fast Food Restaurant,45.536015,-73.568501


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.


In [22]:
## function to repeat the same process to all the neighborhoods in Montreal

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
#run the previous function for all the neighborhoods
montreal_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )



Rivière-des-Prairies
Beaurivage
Tétreaultville
Dupéré
Guybourg
Longue-Pointe
Louis-Riel
Marie-Victorin
Maisonneuve
René-Goupil
Sainte-Lucie
Sault-au-Récollet
Saint-Sulpice
Nicolas-Viel
François-Perrault
Savane
Snowdon
Côte-Saint-Antoine
Ville-Émard
Saint-Henri
Côte-Saint-Paul
Pointe-Saint-Charles
Petite-Bourgogne
Montagne
René-Lévesque
Édouard-Montpetit
Parc-Kent
Hochelaga
Sainte-Marie
Préfontaine
Petite-Côte
Vieux-Rosemont
Étienne Desmarteaux
Louis-Hébert
Lorimier
Saint-Louis
Parc-Lafontaine
Parc-Laurier
Mile End
Gabriel-Sagard
Parc-Extension
Saint-Édouard
Parc-Jarry
Crémazie
Père-Marquette
Pointe-aux-Trembles
Marc-Aurèle-Fortin
Vieux-Montréal
Loyola
Upper Lachine
La Visitation
Nouveau-Bordeaux
Cartierville
Anjou
Outremont
Chameran/Montpellier
Grenet
Dutrisac
Bois-Francs
Est
Centre
Grande-Prairie
Port-Maurice
Desmarchais-Crawford
Verdun-Centre
Ile-des-Soeurs
Sault-Saint-Louis
Cecil-P.-Newman
Lachine-Ouest
Du College/Hodge
Milton-Parc
Pierrefonds-Ouest
Montréal-Est
Westmount
Hampstead


In [24]:
#see the shape and the firsts rows of the dataset
print(montreal_venues.shape)
montreal_venues.head()

(1252, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rivière-des-Prairies,45.648243,-73.544708,"United Rentals - Fluid Solutions: Pumps, Tanks...",45.649669,-73.540676,Construction & Landscaping
1,Beaurivage,45.589751,-73.523954,SAQ,45.585742,-73.522121,Liquor Store
2,Beaurivage,45.589751,-73.523954,Bellepro's,45.591423,-73.525745,Fast Food Restaurant
3,Beaurivage,45.589751,-73.523954,IGA,45.594100,-73.524173,Grocery Store
4,Beaurivage,45.589751,-73.523954,Port De Montréal (terminal Cast),45.590207,-73.520035,Boat or Ferry


In [25]:
print('There are {} uniques categories.'.format(len(montreal_venues['Venue Category'].unique())))

There are 227 uniques categories.


## Analysis of each neighborhoods

In [26]:
# one hot encoding
montreal_onehot = pd.get_dummies(montreal_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
montreal_onehot['Neighborhood'] = montreal_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [montreal_onehot.columns[-1]] + list(montreal_onehot.columns[:-1])
montreal_onehot = montreal_onehot[fixed_columns]

montreal_onehot.head()

,Neighborhood,ATM,African Restaurant,Airport,Airport Terminal,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Rec Center,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Electronics Store,English Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Travel,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hunting Supply,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lebanese Restaurant,Lighthouse,Lingerie Store,Liquor Store,Lounge,Luggage Store,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nightclub,North Indian Restaurant,Office,Organic Grocery,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Sandwich Place,Sausage Shop,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stadium,Stationery Store,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Rivière-des-Prairies,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Beaurivage,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [27]:
montreal_onehot.shape

(1252, 228)

In [28]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
montreal_grouped = montreal_onehot.groupby('Neighborhood').mean().reset_index()
montreal_grouped

,Neighborhood,ATM,African Restaurant,Airport,Airport Terminal,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Rec Center,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Electronics Store,English Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Travel,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hunting Supply,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lebanese Restaurant,Lighthouse,Lingerie Store,Liquor Store,Lounge,Luggage Store,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nightclub,North Indian Restaurant,Office,Organic Grocery,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Sandwich Place,Sausage Shop,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stadium,Stationery Store,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Anjou,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000

In [29]:
montreal_grouped.shape

(88, 228)

In [30]:
#print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in montreal_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = montreal_grouped[montreal_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Anjou----
                  venue  freq
0           Pizza Place  0.25
1  Fast Food Restaurant  0.25
2        Hardware Store  0.25
3                  Park  0.25
4                   ATM  0.00


----Baie-D'Urfé----
                 venue  freq
0    Convenience Store   0.5
1   Italian Restaurant   0.5
2                  ATM   0.0
3      Organic Grocery   0.0
4  Monument / Landmark   0.0


----Beaconsfield----
                    venue  freq
0  Furniture / Home Store   1.0
1                     ATM   0.0
2                  Office   0.0
3       Mobile Phone Shop   0.0
4     Monument / Landmark   0.0


----Beaurivage----
                  venue  freq
0          Liquor Store   0.4
1         Boat or Ferry   0.2
2  Fast Food Restaurant   0.2
3         Grocery Store   0.2
4                   ATM   0.0


----Bois-Francs----
                    venue  freq
0      Athletics & Sports   0.2
1              Print Shop   0.2
2            Hockey Arena   0.2
3  Furniture / Home Store   0.2
4           

In [31]:
#put the results in a dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
#new dataframe containing the 10 best venues of each neighborhoods
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = montreal_grouped['Neighborhood']

for ind in np.arange(montreal_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(montreal_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anjou,Pizza Place,Fast Food Restaurant,Hardware Store,Park,Dessert Shop,Fish Market,Fish & Chips Shop,Farmers Market,Falafel Restaurant,English Restaurant
1,Baie-D'Urfé,Convenience Store,Italian Restaurant,Zoo,Discount Store,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
2,Beaconsfield,Furniture / Home Store,Zoo,Food Truck,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant
3,Beaurivage,Liquor Store,Boat or Ferry,Fast Food Restaurant,Grocery Store,Dive Bar,Food Truck,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop
4,Bois-Francs,Hockey Arena,Print Shop,Furniture / Home Store,Athletics & Sports,Jewelry Store,Diner,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


## Clustering 

In [33]:
# set number of clusters
kclusters = 5

montreal_grouped_clustering = montreal_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(montreal_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 4, 0, 0, 2, 0, 2, 2], dtype=int32)

In [34]:
# new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
montreal_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
montreal_merged = montreal_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

montreal_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rivière-des-Prairies–Pointe-aux-Trembles,Rivière-des-Prairies,45.648243,-73.544708,3.0,Construction & Landscaping,Zoo,Discount Store,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
1,Mercier–Hochelaga-Maisonneuve,Beaurivage,45.589751,-73.523954,4.0,Liquor Store,Boat or Ferry,Fast Food Restaurant,Grocery Store,Dive Bar,Food Truck,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop
2,Mercier–Hochelaga-Maisonneuve,Tétreaultville,45.612298,-73.516443,2.0,Grocery Store,Pharmacy,Breakfast Spot,Diner,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
3,Mercier–Hochelaga-Maisonneuve,Dupéré,45.604545,-73.537125,2.0,Pizza Place,Skating Rink,Hardware Store,Park,Dessert Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
4,Mercier–Hochelaga-Maisonneuve,Guybourg,45.576035,-73.529675,1.0,Gym,Zoo,Diner,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant


In [35]:
print(montreal_merged.dtypes)

Borough                    object
Neighborhood               object
Latitude                  float64
Longitude                 float64
Cluster Labels            float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object


In [36]:
#change type of the column cluster labels
montreal_merged['Cluster Labels'] = montreal_merged['Cluster Labels'].fillna(0).astype('Int64')

In [37]:
montreal_merged.shape

(91, 15)

In [38]:
#drop NaN values
montreal_merged.dropna()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rivière-des-Prairies–Pointe-aux-Trembles,Rivière-des-Prairies,45.648243,-73.544708,3,Construction & Landscaping,Zoo,Discount Store,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
1,Mercier–Hochelaga-Maisonneuve,Beaurivage,45.589751,-73.523954,4,Liquor Store,Boat or Ferry,Fast Food Restaurant,Grocery Store,Dive Bar,Food Truck,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop
2,Mercier–Hochelaga-Maisonneuve,Tétreaultville,45.612298,-73.516443,2,Grocery Store,Pharmacy,Breakfast Spot,Diner,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
3,Mercier–Hochelaga-Maisonneuve,Dupéré,45.604545,-73.537125,2,Pizza Place,Skating Rink,Hardware Store,Park,Dessert Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
4,Mercier–Hochelaga-Maisonneuve,Guybourg,45.576035,-73.529675,1,Gym,Zoo,Diner,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant
5,Mercier–Hochelaga-Maisonneuve,Longue-Pointe,45.588394,-73.524900,4,Liquor Store,Fast Food Restaurant,Boat or Ferry,Zoo,Dive Bar,Food Truck,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop
6,Mercier–Hochelaga-Maisonneuve,Louis-Riel,45.590650,-73.548714,0,Pizza Place,Arts & Crafts Store,Department Store,Shoe Store,Liquor Store,Furniture / Home Store,Sushi Restaurant,Pet Store,Gymnastics Gym,History Museum
7,Rosemont–La Petite-Patrie,Marie-Victorin,45.573449,-73.548974,0,Greek Restaurant,Convenience Store,Breakfast Spot,Pharmacy,Coffee Shop,Bowling Alley,Thai Restaurant,Beer Store,Italian Restaurant,Japanese Restaurant
8,Mercier–Hochelaga-Maisonneuve,Maisonneuve,45.556779,-73.554282,0,Food Truck,Café,Garden,Stadium,Museum,Restaurant,Fast Food Restaurant,Department Store,Thai Restaurant,Scenic Lookout
9,Villeray–Saint-Michel–Parc-Extension,René-Goupil,45.574017,-73.600859,0,Skating Rink,Thai Restaurant,Sandwich Place,Lingerie Store,Supermarket,Asian Restaurant,English Restaurant,Dog Run,Electronics Store,Falafel Restaurant


In [39]:
montreal_merged.shape

(91, 15)

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
colors=['blue','red', 'yellow', 'black', 'brown', 'green']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(montreal_merged['Latitude'], montreal_merged['Longitude'], montreal_merged['Neighborhood'], montreal_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors[cluster-1],
        fill_color=colors[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# See each cluster and the neighborhoods inside

In [41]:
montreal_merged.loc[montreal_merged['Cluster Labels'] == 0, montreal_merged.columns[[1] + list(range(5, montreal_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Louis-Riel,Pizza Place,Arts & Crafts Store,Department Store,Shoe Store,Liquor Store,Furniture / Home Store,Sushi Restaurant,Pet Store,Gymnastics Gym,History Museum
7,Marie-Victorin,Greek Restaurant,Convenience Store,Breakfast Spot,Pharmacy,Coffee Shop,Bowling Alley,Thai Restaurant,Beer Store,Italian Restaurant,Japanese Restaurant
8,Maisonneuve,Food Truck,Café,Garden,Stadium,Museum,Restaurant,Fast Food Restaurant,Department Store,Thai Restaurant,Scenic Lookout
9,René-Goupil,Skating Rink,Thai Restaurant,Sandwich Place,Lingerie Store,Supermarket,Asian Restaurant,English Restaurant,Dog Run,Electronics Store,Falafel Restaurant
10,Sainte-Lucie,Pizza Place,Gym Pool,Fast Food Restaurant,Liquor Store,Furniture / Home Store,Café,Restaurant,Office,Coffee Shop,Creperie
12,Saint-Sulpice,Fast Food Restaurant,Ice Cream Shop,Coffee Shop,Athletics & Sports,Zoo,Discount Store,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop
13,Nicolas-Viel,Bakery,Italian Restaurant,Breakfast Spot,Fast Food Restaurant,Electronics Store,Storage Facility,Movie Theater,Gas Station,Music Venue,Restaurant
14,François-Perrault,Pharmacy,Liquor Store,Clothing Store,Discount Store,Supermarket,Sushi Restaurant,Bar,Seafood Restaurant,BBQ Joint,Gym / Fitness Center
15,Savane,Cambodian Restaurant,Clothing Store,Paper / Office Supplies Store,Bagel Shop,Diner,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
19,Saint-Henri,Breakfast Spot,Café,Coffee Shop,Thai Restaurant,Boutique,French Restaurant,Bike Shop,Speakeasy,English Restaurant,Gastropub


In [42]:
montreal_merged.loc[montreal_merged['Cluster Labels'] == 1, montreal_merged.columns[[1] + list(range(5, montreal_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Guybourg,Gym,Zoo,Diner,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant
75,Côte-Saint-Luc,Gym,Burrito Place,Zoo,Diner,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [43]:
montreal_merged.loc[montreal_merged['Cluster Labels'] == 2, montreal_merged.columns[[1] + list(range(5, montreal_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Tétreaultville,Grocery Store,Pharmacy,Breakfast Spot,Diner,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
3,Dupéré,Pizza Place,Skating Rink,Hardware Store,Park,Dessert Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
11,Sault-au-Récollet,Pizza Place,Pharmacy,Convenience Store,Metro Station,Discount Store,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant
16,Snowdon,Moving Target,Museum,Zoo,Food & Drink Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant
17,Côte-Saint-Antoine,Breakfast Spot,Thrift / Vintage Store,Zoo,Diner,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
18,Ville-Émard,Pool,Zoo,Diner,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant
20,Côte-Saint-Paul,Pizza Place,Convenience Store,Park,Grocery Store,Fast Food Restaurant,Breakfast Spot,Sushi Restaurant,Restaurant,Gourmet Shop,Metro Station
26,Parc-Kent,Park,Pharmacy,Sandwich Place,Gym,Diner,Café,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
28,Sainte-Marie,Bike Rental / Bike Share,Liquor Store,Park,Grocery Store,Zoo,Discount Store,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant
30,Petite-Côte,Park,Metro Station,Bus Line,Pet Store,Convenience Store,Bus Stop,Pharmacy,Harbor / Marina,Department Store,Falafel Restaurant


In [44]:
montreal_merged.loc[montreal_merged['Cluster Labels'] == 3, montreal_merged.columns[[1] + list(range(5, montreal_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rivière-des-Prairies,Construction & Landscaping,Zoo,Discount Store,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
45,Pointe-aux-Trembles,Construction & Landscaping,Zoo,Discount Store,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [45]:
montreal_merged.loc[montreal_merged['Cluster Labels'] == 4, montreal_merged.columns[[1] + list(range(5, montreal_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Beaurivage,Liquor Store,Boat or Ferry,Fast Food Restaurant,Grocery Store,Dive Bar,Food Truck,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop
5,Longue-Pointe,Liquor Store,Fast Food Restaurant,Boat or Ferry,Zoo,Dive Bar,Food Truck,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop
21,Pointe-Saint-Charles,Boat or Ferry,Zoo,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,English Restaurant
